In [1]:
from datetime import datetime, timedelta, time
from glob import glob
import pyotp
import configparser
from smartapi import SmartConnect
import pandas as pd
import numpy as np
from datetime import *
import re
from time import sleep, strptime
from time import time as tim
import requests

config = configparser.ConfigParser()

# credentials
token = 'S3N5FR3JZ6OSTM3PBPHJRNWEP4'
config.read('my_credentials.ini')
# token = 'OTXOCWZA4CU4YP7LUUZCZLOCYM'
# config.read('shahabas_credentials.ini')
client_id = config['angel']['client_id']
password = config['angel']['password']
api_key = config['angel']['api_key']

def intializeSymbolTokenMap():
    url = 'https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json'
    d = requests.get(url).json()
    global token_df
    token_df = 'test inside fun'
    token_df = pd.DataFrame.from_dict(d)
    token_df['expiry'] = pd.to_datetime(token_df['expiry'])
    token_df = token_df.astype({'strike': float})

# fetch token/instrument from dataframe
def getTokenInfo (symbol,exch_seg='NSE',instrumenttype='OPTIDX',strike_price='',pe_ce='CE',expiry=None):
    df = token_df
    strike_price = strike_price*100
    if exch_seg == 'NSE':
        eq_df = df[(df['exch_seg'] == 'NSE') ]
        return eq_df[eq_df['name'] == symbol]
    elif exch_seg == 'NFO' and ((instrumenttype == 'FUTSTK') or (instrumenttype == 'FUTIDX')):
        return df[(df['exch_seg'] == 'NFO') & (df['instrumenttype'] == instrumenttype) & (df['name'] == symbol)].sort_values(by=['expiry'])
    elif exch_seg == 'NFO' and (instrumenttype == 'OPTSTK' or instrumenttype == 'OPTIDX'):
        return df[(df['exch_seg'] == 'NFO') & (df['instrumenttype'] == instrumenttype) & (df['name'] == symbol) & (df['strike'] == strike_price) & (df['symbol'].str.endswith(pe_ce))].sort_values(by=['expiry'])

intializeSymbolTokenMap()
obj=SmartConnect(api_key=api_key)
data = obj.generateSession(client_id,password,pyotp.TOTP(token).now())
refreshToken= data['data']['refreshToken']
feedToken=obj.getfeedToken()
userProfile= obj.getProfile(refreshToken)
print(userProfile)

{'status': True, 'message': 'SUCCESS', 'errorcode': '', 'data': {'clientcode': 'A471478', 'name': 'ABDULLA NISHAD', 'email': '', 'mobileno': '', 'exchanges': ['bse_cm', 'nse_cm', 'nse_fo'], 'products': ['BO', 'NRML', 'CO', 'CNC', 'MIS', 'MARGIN'], 'lastlogintime': '', 'broker': ''}}


In [83]:
start = datetime.now()

try:
    final_df = pd.read_csv('updated_bnf_spot.csv')
    final_df['datetime'] = pd.to_datetime(final_df['datetime'])
except:
    from_ = 0
    final_df = pd.DataFrame()
    pass

from_date = str(final_df.iloc[-1].datetime)[:-3]
to_date = str(final_df.iloc[-1].datetime + timedelta(days=1))[:-3]
to_date

while from_date < str(datetime.now()):
    historicParam = {
         "exchange": "NFO",
         "symboltoken": 'BANKNIFTY',
         "interval": "ONE_MINUTE",
         "fromdate": from_date,
         "todate": to_date
    }
    x = obj.getCandleData(historicParam)
    
    # clean datetime str in list itself before moving to df
    if x['data'] != None:
        data = [[i[0].replace('T',' ')[:-6]] + i[1:] for i in x['data']]
    
        # creating and concatinating df with final df
        df = pd.DataFrame(data,columns=['datetime','open','high','low','close','volume'])
        df['datetime'] = pd.to_datetime(df['datetime'])
        final_df = pd.concat([final_df,df],axis=0)
    
    print(from_date)
    from_date = to_date
    to_date = str(datetime.strptime(to_date,"%Y-%m-%d %H:%M") + timedelta(days=1))[:-3]

final_df.drop_duplicates(inplace=True)
final_df.to_csv('updated_bnf_spot.csv', index=False)